# Assignment 2 : Inconsistancy Rate calculation #

1. Used status columns from station_a, station_b, station_c and level_cm of station_main
2. Used SAX 
3. used "level_cm" of station_main as reference and binning
4. Followed Exercise Solution 04
5. Applied the IR calculation methodology from the lecture note (chapter 4, slide 75)
6. Calculated IR = 0.26477860839562967

In [132]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import string

col_time= "time"
col_temp_c = "temp_c"
col_status = "status"
col_rain_mm = "rain_mm"
col_level_cm = "level_cm"
col_flow_m2_s = "flow_m2_s"

statin_a_csv = "station_a_improved.csv"
statin_b_csv = "station_b_improved.csv"
statin_c_csv = "station_c_improved.csv"
station_main_csv = "station_main_improved.csv"



## Implementation of the SAX (Symbolic Aggregate Approximation) Algorithm

In [133]:
class SAX(object):
    """
    This class is for computing common things with the Symbolic
    Aggregate approXimation method.  In short, this translates
    a series of data to a string, which can then be compared with other
    such strings using a lookup table.
    """

    def __init__(self, wordSize = 9, alphabetSize = 10, epsilon = 1e-6):

        if alphabetSize < 3 or alphabetSize > 20:
            raise DictionarySizeIsNotSupported()
        self.aOffset = ord('a')
        self.wordSize = wordSize
        self.alphabetSize = alphabetSize
        self.eps = epsilon
        self.breakpoints = {'3' : [-0.43, 0.43],
                            '4' : [-0.67, 0, 0.67],
                            '5' : [-0.84, -0.25, 0.25, 0.84],
                            '6' : [-0.97, -0.43, 0, 0.43, 0.97],
                            '7' : [-1.07, -0.57, -0.18, 0.18, 0.57, 1.07],
                            '8' : [-1.15, -0.67, -0.32, 0, 0.32, 0.67, 1.15],
                            '9' : [-1.22, -0.76, -0.43, -0.14, 0.14, 0.43, 0.76, 1.22],
                            '10': [-1.28, -0.84, -0.52, -0.25, 0, 0.25, 0.52, 0.84, 1.28],
                            '11': [-1.34, -0.91, -0.6, -0.35, -0.11, 0.11, 0.35, 0.6, 0.91, 1.34],
                            '12': [-1.38, -0.97, -0.67, -0.43, -0.21, 0, 0.21, 0.43, 0.67, 0.97, 1.38],
                            '13': [-1.43, -1.02, -0.74, -0.5, -0.29, -0.1, 0.1, 0.29, 0.5, 0.74, 1.02, 1.43],
                            '14': [-1.47, -1.07, -0.79, -0.57, -0.37, -0.18, 0, 0.18, 0.37, 0.57, 0.79, 1.07, 1.47],
                            '15': [-1.5, -1.11, -0.84, -0.62, -0.43, -0.25, -0.08, 0.08, 0.25, 0.43, 0.62, 0.84, 1.11, 1.5],
                            '16': [-1.53, -1.15, -0.89, -0.67, -0.49, -0.32, -0.16, 0, 0.16, 0.32, 0.49, 0.67, 0.89, 1.15, 1.53],
                            '17': [-1.56, -1.19, -0.93, -0.72, -0.54, -0.38, -0.22, -0.07, 0.07, 0.22, 0.38, 0.54, 0.72, 0.93, 1.19, 1.56],
                            '18': [-1.59, -1.22, -0.97, -0.76, -0.59, -0.43, -0.28, -0.14, 0, 0.14, 0.28, 0.43, 0.59, 0.76, 0.97, 1.22, 1.59],
                            '19': [-1.62, -1.25, -1, -0.8, -0.63, -0.48, -0.34, -0.2, -0.07, 0.07, 0.2, 0.34, 0.48, 0.63, 0.8, 1, 1.25, 1.62],
                            '20': [-1.64, -1.28, -1.04, -0.84, -0.67, -0.52, -0.39, -0.25, -0.13, 0, 0.13, 0.25, 0.39, 0.52, 0.67, 0.84, 1.04, 1.28, 1.64]
                            }
        self.beta = self.breakpoints[str(self.alphabetSize)]
        self.build_letter_compare_dict()
        self.scalingFactor = 1


    def to_letter_rep(self, x):
        """
        Function takes a series of data, x, and transforms it to a string representation
        """
        (paaX, indices) = self.to_PAA(self.normalize(x))
        self.scalingFactor = np.sqrt(len(x) / self.wordSize)
        return (self.alphabetize(paaX), indices)

    def normalize(self, x):
        """
        Function will normalize an array (give it a mean of 0, and a
        standard deviation of 1) unless it's standard deviation is below
        epsilon, in which case it returns an array of zeros the length
        of the original array.
        """
        X = np.asanyarray(x)
        if X.std() < self.eps:
            return [0 for entry in X]
        return (X-X.mean())/X.std()

    def to_PAA(self, x):
        """
        Funciton performs Piecewise Aggregate Approximation on data set, reducing
        the dimension of the dataset x to w discrete levels. returns the reduced
        dimension data set, as well as the indicies corresponding to the original
        data for each reduced dimension
        """
        n = len(x)
        stepFloat = n/float(self.wordSize)
        step = int(math.ceil(stepFloat))
        frameStart = 0
        approximation = []
        indices = []
        i = 0
        while frameStart <= n-step:
            thisFrame = np.array(x[frameStart:int(frameStart + step)])
            approximation.append(np.mean(thisFrame))
            indices.append((frameStart, int(frameStart + step)))
            i += 1
            frameStart = int(i*stepFloat)
        return (np.array(approximation), indices)

    def alphabetize(self,paaX):
        """
        Converts the Piecewise Aggregate Approximation of x to a series of letters.
        """
        alphabetizedX = ''
        for i in range(0, len(paaX)):
            letterFound = False
            for j in range(0, len(self.beta)):
                if paaX[i] < self.beta[j]:
                    alphabetizedX += chr(self.aOffset + j)
                    letterFound = True
                    break
            if not letterFound:
                alphabetizedX += chr(self.aOffset + len(self.beta))
        return alphabetizedX

    def compare_strings(self, sA, sB, x=None):
        """
        Compares two strings based on individual letter distance
        Requires that both strings are the same length
        """
        
        if x != None:
            self.scalingFactor = np.sqrt(x / self.wordSize)
        
        if len(sA) != len(sB):
            raise StringsAreDifferentLength()
        list_letters_a = [x for x in sA]
        list_letters_b = [x for x in sB]
        mindist = 0.0
        for i in range(0, len(list_letters_a)):
            mindist += self.compare_letters(list_letters_a[i], list_letters_b[i])**2
        mindist = self.scalingFactor* np.sqrt(mindist)
        return mindist

    def compare_letters(self, la, lb):
        """
        Compare two letters based on letter distance return distance between
        """
        return self.compareDict[la+lb]

    def build_letter_compare_dict(self):
        """
        Builds up the lookup table to determine numeric distance between two letters
        given an alphabet size.  Entries for both 'ab' and 'ba' will be created
        and will have identical values.
        """

        number_rep = range(0,self.alphabetSize)
        letters = [chr(x + self.aOffset) for x in number_rep]
        self.compareDict = {}
        for i in range(0, len(letters)):
            for j in range(0, len(letters)):
                if np.abs(number_rep[i]-number_rep[j]) <=1:
                    self.compareDict[letters[i]+letters[j]] = 0
                else:
                    high_num = np.max([number_rep[i], number_rep[j]])-1
                    low_num = np.min([number_rep[i], number_rep[j]])
                    self.compareDict[letters[i]+letters[j]] = self.beta[high_num] - self.beta[low_num]

    def sliding_window(self, x, numSubsequences = None, overlappingFraction = None):
        if not numSubsequences:
            numSubsequences = 20
        self.windowSize = int(len(x)/numSubsequences)
        if not overlappingFraction:
            overlappingFraction = 0.9
        overlap = self.windowSize*overlappingFraction
        moveSize = int(self.windowSize - overlap)
        if moveSize < 1:
            raise OverlapSpecifiedIsNotSmallerThanWindowSize()
        ptr = 0
        n = len(x)
        windowIndices = []
        stringRep = []
        while ptr < n-self.windowSize+1:
            thisSubRange = x[ptr:ptr+self.windowSize]
            (thisStringRep,indices) = self.to_letter_rep(thisSubRange)
            stringRep.append(thisStringRep)
            windowIndices.append((ptr, ptr+self.windowSize))
            ptr += moveSize
        return (stringRep,windowIndices)
    
    def batch_compare(self, xStrings, refString):
        return [self.compare_strings(x, refString) for x in xStrings]

    def set_scaling_factor(self, scalingFactor):
        self.scalingFactor = scalingFactor

    def set_window_size(self, windowSize):
        self.windowSize = windowSize


In [134]:
# Nothing to do here

def plot_SAX(timeSeries, sax):
     
    """Helper Method to plot final result, 
    requires a dataFrame with columns paa1 and paa2 and a SAX object."""

    timeSeries.plot(y=["paa_a", "paa_b","paa_c"], use_index=True)
    alphabet = list(string.ascii_lowercase)
    segments = sax.breakpoints[str(sax.alphabetSize)]

    for i in range(len(segments)):
        plt.axhline(segments[i], color='r', linestyle='--', label=alphabet[i])

    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

### Prepare Data ##

In [135]:
station_a = pd.read_csv(statin_a_csv, parse_dates=['time'],index_col='time')
station_a.rename(columns={col_temp_c: "a_temp", col_status: "a_status", col_rain_mm: "a_rain" },inplace=True)
station_a = station_a.loc[:, ~station_a.columns.str.contains('^Unnamed')]

station_b = pd.read_csv(statin_b_csv, parse_dates=[col_time],index_col=col_time)
station_b.rename(columns={col_temp_c: "b_temp", col_status: "b_status", col_rain_mm: "b_rain" },inplace=True)
station_b = station_b.loc[:, ~station_b.columns.str.contains('^Unnamed')]

station_c = pd.read_csv(statin_c_csv, parse_dates=[col_time],index_col=col_time)
station_c.rename(columns={col_temp_c: "c_temp", col_status: "c_status", col_rain_mm: "c_rain" },inplace=True)
station_c = station_c.loc[:, ~station_c.columns.str.contains('^Unnamed')]

station_main = pd.read_csv(station_main_csv, parse_dates=[col_time],index_col=col_time)
station_main = station_main.loc[:, ~station_main.columns.str.contains('^Unnamed')]

station_main_water_level =  station_main.drop([col_flow_m2_s], axis = 1) 
station_main_water_level[col_level_cm][station_main_water_level[col_level_cm]<0] = 0

merged_sation_a_b = pd.merge(left=station_a, right=station_b, left_on='time', right_on='time')
merged_station_a_b_c = pd.merge(left=merged_sation_a_b, right=station_c, left_on='time', right_on='time')

status = merged_station_a_b_c.drop(['a_temp', 'a_rain','b_temp', 'b_rain','c_temp', 'c_rain'], axis = 1) 

status_water_level = pd.merge(left=status, right=station_main_water_level, left_on='time', right_on='time')

### Make Bins

In [136]:
sax = SAX()

In [137]:
standarize_level = "standarize_level"
col_word_level = "word_level"
status_water_level[standarize_level] = sax.normalize(status_water_level[col_level_cm])
status_water_level[col_word_level] = list(sax.alphabetize(status_water_level[standarize_level].tolist()))
# status_water_level.groupby("word_level").count()

unique_classes = status_water_level[col_word_level].unique()
unique_classes


array(['d', 'e', 'f', 'g', 'h', 'i', 'j'], dtype=object)

### Classes are ##

['d', 'e', 'f', 'g', 'h', 'i', 'j']

### fill up data inside according to classes ##

In [138]:
# For unique tuple
samples = set()
# Create lookup dictionary for class count for sample.
class_dict = {}

for index, row in status_water_level.iterrows():
    sample = (row["a_status"], row["b_status"], row["c_status"])
    class_dict[sample] = np.zeros(len(uniqe_wordBins))

# Count occurences of words

for index, row in status_water_level.iterrows():
    sample = (row["a_status"], row["b_status"], row["c_status"])
    try:
        value_index = np.where(uniqe_wordBins ==row[col_word_level])
        class_dict[sample][value_index] += 1
    except Exception as ex:
        print(ex)



### Calculate IZs and IR ##

In [139]:
izs = [] 
for sample in class_dict:
    iz = np.sum(class_dict[sample]) - np.max(class_dict[sample])
    izs.append(iz)

IR = sum(izs) / len(status_water_level)
IR

0.26477860839562967